In [2]:
import debugpy
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import ListedColormap
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
import os
import gzip
import numpy as np
import cell2location

[rank: 0] Global seed set to 0


In [4]:
# The model and output h5ad can be loaded later like this:
results_folder = './out/cell2loc/'

# create paths and names to results folders for reference regression and cell2location models
ref_run_name = f'{results_folder}/reference_signatures'
run_name = f'{results_folder}/cell2location_map'


adata_file = f"{ref_run_name}/sc.h5ad"
adata_ref = sc.read_h5ad(adata_file)

In [25]:
inf_aver = adata_ref.varm['means_per_cluster_mu_fg'][[f'means_per_cluster_mu_fg_{i}'
                                    for i in adata_ref.uns['mod']['factor_names']]].copy()
inf_aver.columns = adata_ref.uns['mod']['factor_names']
inf_aver.iloc[0:5, 0:5]

,fibroblast,ependymal cell,endothelial cell,astrocyte,oligodendrocyte
ENSMUSG00000051951,0.001433,0.000771,0.000077,0.000186,0.005343
ENSMUSG00000103377,0.000602,0.000649,0.000062,0.000029,0.001967
ENSMUSG00000104017,0.000849,0.001388,0.000075,0.000019,0.001560
ENSMUSG00000089699,0.000395,0.000240,0.000053,0.000016,0.000017
ENSMUSG00000103161,0.000335,0.001076,0.000082,0.000029,0.000790


In [6]:
def calcQCmat(andata):
    andata.var_names_make_unique()
    andata.var["mt"] = andata.var_names.str.startswith("mt-")
    andata.var["ribo"] = andata.var_names.str.startswith(("RPS", "RPL"))
    andata.var["hb"] = andata.var_names.str.contains("^HB[^(P)]")
    sc.pp.calculate_qc_metrics(andata, qc_vars=["mt", "ribo", "hb"], inplace=True, log1p=True)
    return andata
path_016 = "/data/kanferg/Sptial_Omics/playGround/Data/Visium_HD_Mouse_Brain_square_example/square_016um"
andata016_ = sc.read_visium(path=path_016)
andata016 = calcQCmat(andata016_)
andata016 = andata016[:,andata016.var.n_cells_by_counts > 50]
andata016.raw = andata016.copy()
sc.pp.normalize_total(andata016)
sc.pp.log1p(andata016)
log1p_data = andata016.X.todense()
sc.pp.highly_variable_genes(andata016)
sc.pp.scale(andata016)
andata016.obsm['spatial'] = np.array(andata016.obsm['spatial'], dtype=np.float64)
sc.pp.pca(andata016, n_comps=10)
sc.pp.neighbors(andata016)
sc.tl.leiden(andata016, key_added="clusters" ,resolution=0.7 , directed=False, n_iterations=2)

/usr/local/apps/cell2location/conda/envs/0.1.3/lib/python3.9/site-packages/anndata/_core/anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/usr/local/apps/cell2location/conda/envs/0.1.3/lib/python3.9/site-packages/scanpy/readwrite.py:413: DtypeWarning: Columns (1,2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  positions = pd.read_csv(files['tissue_positions_file'], header=None)


In [7]:
andata016.X = andata016.raw.X

In [15]:
andata016.var.set_index('gene_ids', drop=True, inplace=True)
andata016.var

,feature_types,genome,mt,ribo,hb,n_cells_by_counts,mean_counts,log1p_mean_counts,pct_dropout_by_counts,total_counts,log1p_total_counts,highly_variable,means,dispersions,dispersions_norm,mean,std
gene_ids,,,,,,,,,,,,,,,,,
ENSMUSG00000051951,Gene Expression,mm10,False,False,False,4837,0.053176,0.051810,95.110042,5260.0,8.568076,False,0.032296,0.075234,-1.202811,0.023201,0.117684
ENSMUSG00000025902,Gene Expression,mm10,False,False,False,709,0.007885,0.007854,99.283237,780.0,6.660575,False,0.008487,0.649225,1.555923,0.005139,0.067161
ENSMUSG00000025903,Gene Expression,mm10,False,False,False,2083,0.021624,0.021394,97.894194,2139.0,7.668561,False,0.015538,0.221813,-0.498320,0.010742,0.084022
ENSMUSG00000033813,Gene Expression,mm10,False,False,False,11157,0.127268,0.119797,88.720847,12589.0,9.440659,False,0.085888,0.231577,-0.451390,0.060431,0.196228
ENSMUSG00000002459,Gene Expression,mm10,False,False,False,4643,0.049658,0.048464,95.306166,4912.0,8.499640,False,0.040883,0.425136,0.478900,0.027108,0.140343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSMUSG00000064363,Gene Expression,mm10,True,False,False,74014,1.752489,1.012506,25.175652,173351.0,12.063080,False,1.023667,1.000293,0.285638,0.800459,0.641889
ENSMUSG00000064367,Gene Expression,mm10,True,False,False,75405,1.894366,1.062766,23.769423,187385.0,12.140926,False,1.045383,0.918470,-0.907876,0.830585,0.637000
ENSMUSG00000064368,Gene Expression,mm10,True,False,False,20290,0.239190,0.214458,79.487853,23660.0,10.071584,True,0.234762,0.718311,1.495231,0.151443,0.348762


In [27]:
# find shared genes and subset both anndata and reference signatures
intersect = np.intersect1d(andata016.var_names, inf_aver.index)
adata_vis = andata016[:, intersect].copy()
inf_aver = inf_aver.loc[intersect, :].copy()

In [29]:
# prepare anndata for cell2location model
cell2location.models.Cell2location.setup_anndata(adata=adata_vis, batch_key=None)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [35]:
# create and train the model
mod = cell2location.models.Cell2location(
    adata_vis, cell_state_df=inf_aver,
    # the expected average cell abundance: tissue-dependent
    # hyper-prior which can be estimated from paired histology:
    N_cells_per_location=20,
    # hyperparameter controlling normalisation of
    # within-experiment variation in RNA detection:
    detection_alpha=200
)
mod.view_anndata_setup()

Anndata setup with scvi-tools version 0.20.3.

Setup via `Cell2location.setup_anndata` with arguments:

{
│   'layer': None,
│   'batch_key': None,
│   'labels_key': None,
│   'categorical_covariate_keys': None,
│   'continuous_covariate_keys': None
}

         Summary Statistics         
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃     Summary Stat Key     ┃ Value ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│         n_batch          │   1   │
│         n_cells          │ 98917 │
│ n_extra_categorical_covs │   0   │
│ n_extra_continuous_covs  │   0   │
│         n_labels         │   1   │
│          n_vars          │ 12030 │
└──────────────────────────┴───────┘

               Data Registry                
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Registry Key ┃    scvi-tools Location    ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      X       │          adata.X          │
│    batch     │ adata.obs['_scvi_batch']  │
│    ind_x     │   adata.obs['_indices']   │
│    labels    │ adata.obs['_scvi_labels'] │
└──────────────┴───────────────────────────┘

                     batch State Registry                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃     Source Location      ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['_scvi_batch'] │     0      │          0          │
└──────────────────────────┴────────────┴─────────────────────┘

                     labels State Registry                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃      Source Location      ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['_scvi_labels'] │     0      │          0          │
└───────────────────────────┴────────────┴─────────────────────┘

In [ ]:
mod.train(max_epochs=3000,
          # train using full data (batch_size=None)
          batch_size=3000,
          # use all data points in training because
          # we need to estimate cell abundance at all locations
          train_size=1,
          use_gpu=False,
         )

# plot ELBO loss history during training, removing first 100 epochs from the plot
mod.plot_history(1000)
plt.legend(labels=['full data training']);

/usr/local/apps/cell2location/conda/envs/0.1.3/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /usr/local/apps/cell2location/conda/envs/0.1.3/lib/p ...
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/apps/cell2location/conda/envs/0.1.3/lib/python3.9/site-packages/pytorch_lightning/trainer/setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=4)`.
  rank_zero_warn(
/usr/local/apps/cell2location/conda/envs/0.1.3/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on y

Epoch 37/3000:   1%|▎                            | 36/3000 [2:26:22<204:44:51, 248.68s/it, v_num=1, elbo_train=2.55e+8]